In [20]:
import matplotlib.pyplot as plt
import snapatac2 as snap
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.metrics import silhouette_score
import glob
import warnings
import os
warnings.filterwarnings("ignore")


In [21]:
df_l3_summary = pd.read_csv("/data2st1/junyi/output/atac0416/iterative/l3/l3_summary.csv")

In [29]:
df_l4_summary = pd.read_csv("/data2st1/junyi/output/atac0416/iterative/l4/l4_summary.csv",header=None,index_col=0)

In [23]:
df_l3metas= glob.glob("/data2st1/junyi/output/atac0416/iterative/l3/*l3.csv")

In [24]:
df_l4metas = glob.glob("/data2st1/junyi/output/atac0416/iterative/l4/*l4.csv")

In [31]:
df_l4_summary.columns = ['celltype.L3','best_resolution','n_clusters','n_cells']

In [32]:
df_l4_summary['cellpclust'] = df_l4_summary['n_cells']/df_l4_summary['n_clusters']

In [34]:
df_l3 = pd.DataFrame()
for f in df_l3metas:
    df = pd.read_csv(f,index_col=0)
    df_l3 = pd.concat([df_l3, df], axis=0)

In [35]:
df_l4 = pd.DataFrame()
for f in df_l4metas:
    df = pd.read_csv(f,index_col=0)
    df_l4 = pd.concat([df_l4, df], axis=0)

In [55]:
df_merge = df_l3.merge(df_l4,left_index=True, right_index=True,how='left')

In [37]:
grouped = df_merge.groupby(['celltype.L3', 'celltype.L4']).size().reset_index(name='count')

In [38]:
filtered = grouped[grouped['count'] < 200]


In [40]:
filtered.to_csv("/data2st1/junyi/output/atac0416/iterative/l3l4_SMALL.csv",index=False)

In [41]:
df_ncluster = df_merge.groupby(['celltype.L3']).nunique()

In [43]:
df_l200_cluster = filtered.groupby(['celltype.L3']).count()
df_l200_cluster


,celltype.L4,count
celltype.L3,,
AMY Adamtsl1 Glut-0,1,1
AMY Adamtsl1 Glut-1,1,1
AMY Adamtsl1 Glut-3,1,1
AMY Dach1 Glut-1,1,1
AMY Dach1 Glut-2,1,1
AMY Sim1 Glut-0,2,2
AMY Slc17a6 Glut-1,2,2
AMY Sox6 GABA-0,1,1
AMY Sox6 GABA-1,1,1


In [44]:
df_l200_summary = df_l200_cluster.merge(df_ncluster, left_index=True, right_index=True, how='left')

In [45]:
df_l200_summary.columns = ['n_clus<200', 'n_clus<=200','n_clus']

In [ ]:
#Hippo Mossy cell 0 is proproblematic

In [ ]:
# 24 clusters are merged
refine_map = {
    'AMY Dach1 Glut-1-2': 'AMY Dach1 Glut-1-1',
    'AMY Dach1 Glut-2-1': 'AMY Dach1 Glut-2-0',
    'PFC L2/3 IT-1-1': 'PFC L2/3 IT-1-0',
    'AMY Sim1 Glut-0-1':'AMY Sim1 Glut-0-0',
    'AMY Sim1 Glut-0-2':'AMY Sim1 Glut-0-0',
    'AMY Slc17a6 Glut-1-4':'AMY Slc17a6 Glut-1-0',
    'AMY Slc17a6 Glut-1-5':'AMY Slc17a6 Glut-1-2',
    'AMY Sox6 GABA-1-2':'AMY Sox6 GABA-1-1',
    'AMY Zfp536 GABA-0-1':'AMY Zfp536 GABA-0-0',
    'Hippo CA1-v pyramidal-0-1':'Hippo CA1-v pyramidal-0-0',
    'Hippo CA1-v pyramidal-1-1':'Hippo CA1-v pyramidal-1-0',
    'Hippo CA3-d pyramidal-1-1':'Hippo CA3-d pyramidal-1-0',
    'Hippo DG GC-1-3':'Hippo DG GC-1-0',
    'Hippo DG GC-1-4':'Hippo DG GC-1-0',
    'Hippo Lamp5 GABA-0-1':'Hippo Lamp5 GABA-0-0',
    'Hippo Pvalb GABA-0-1':'Hippo Pvalb GABA-0-0',
    'Hippo Sst GABA-0-1':'Hippo Sst GABA-0-0',
    'Hippo Sst GABA-0-2':'Hippo Sst GABA-0-0',
    'PFC L2/3 IT-1-1':'PFC L2/3 IT-1-0',
    'PFC L6 CT-0-8':'PFC L6 CT-0-0',
    'PFC L6 IT-0-2':'PFC L6 IT-0-0',
    'PFC Lamp5 GABA-0-1':'PFC Lamp5 GABA-0-0',
    'PFC NP-1-1':'PFC NP-1-0',
    'PFC Sst GABA-0-1':'PFC Sst GABA-0-0'
    #'Hippo DG GC-1-8':"Hippo DG GC-1-0", Different
    #'PFC L6 IT-0-2':'PFC L6 IT-0-2',#Not move, different
    # 'PFC L6 CT-0-9':'PFC L6 CT-0-3',
    # 'AMY Adamtsl1 Glut-2-5':'AMY Adamtsl1 Glut-2-0',
    # 'AMY Adamtsl1 Glut-2-4':'AMY Adamtsl1 Glut-2-2',
    # 'Hippo CA1-v pyramidal-1-1':'Hippo CA1-v pyramidal-1-0',
    # 'Hippo CA1-v pyramidal-0-1':'Hippo CA1-v pyramidal-0-1',
    # 'PFC Lamp5 GABA-0-1':'PFC Lamp5 GABA-0-0',
    # 'AMY Dach1 Glut-1-3':'AMY Dach1 Glut-1-0',
    # 'AMY Sox6 GABA-0-2':'AMY Sox6 GABA-0-0',
    # 'PFC L6 CT-0-8':'PFC L6 CT-0-0',
    # 'AMY Adamtsl1 Glut-3-1':'AMY Adamtsl1 Glut-3-0',
    # 'AMY Sox6 GABA-1-2':'AMY Sox6 GABA-1-1',
    # 'Hippo Lamp5 GABA-0-1':'Hippo Lamp5 GABA-0-0',
    # 'Hippo CA3-d pyramidal-1-2':'Hippo CA3-d pyramidal-1-0'

}

In [ ]:
df_merge['celltype.L4.raw'] = df_merge['celltype.L4']



In [65]:
df_merge['celltype.L4']=df_merge['celltype.L4'].fillna(df_merge['celltype.L3']+"-0")

In [66]:
df_merge

,celltype.L3,celltype.L4,celltype.L4.raw
MC39C_HIP:AAAGATGAGAACAGGA-1,Hippo Pvalb GABA-0,Hippo Pvalb GABA-0-0,Hippo Pvalb GABA-0-0
MC39C_HIP:AAAGGATGTCCTATTT-1,Hippo Pvalb GABA-0,Hippo Pvalb GABA-0-1,Hippo Pvalb GABA-0-1
MC39C_HIP:AAAGGGCGTTCGGGAA-1,Hippo Pvalb GABA-0,Hippo Pvalb GABA-0-1,Hippo Pvalb GABA-0-1
MC39C_HIP:AACAGTCCATCCCTTG-1,Hippo Pvalb GABA-1,Hippo Pvalb GABA-1-0,NaN
MC39C_HIP:AACGTACCATCCTGGG-1,Hippo Pvalb GABA-0,Hippo Pvalb GABA-0-0,Hippo Pvalb GABA-0-0
...,...,...,...
MW51A_AMY:TTTGGTTCAATCACTG-1,AMY Adamtsl1 Glut-0,AMY Adamtsl1 Glut-0-0,AMY Adamtsl1 Glut-0-0
MW51A_AMY:TTTGGTTGTTGCAGAG-1,AMY Adamtsl1 Glut-2,AMY Adamtsl1 Glut-2-0,AMY Adamtsl1 Glut-2-0
MW51A_AMY:TTTGGTTTCTTAAGCG-1,AMY Adamtsl1 Glut-1,AMY Adamtsl1 Glut-1-0,AMY Adamtsl1 Glut-1-0
MW51A_AMY:TTTGTGTCAAAGCTTC-1,AMY Adamtsl1 Glut-3,AMY Adamtsl1 Glut-3-0,AMY Adamtsl1 Glut-3-0


In [67]:
df_merge['celltype.L4'] = df_merge['celltype.L4'].replace(refine_map)

In [72]:
len(df_merge['celltype.L3'].unique())

98

In [73]:
len(df_merge['celltype.L4'].unique())

160

In [75]:
df_merge

,celltype.L3,celltype.L4,celltype.L4.raw
MC39C_HIP:AAAGATGAGAACAGGA-1,Hippo Pvalb GABA-0,Hippo Pvalb GABA-0-0,Hippo Pvalb GABA-0-0
MC39C_HIP:AAAGGATGTCCTATTT-1,Hippo Pvalb GABA-0,Hippo Pvalb GABA-0-0,Hippo Pvalb GABA-0-1
MC39C_HIP:AAAGGGCGTTCGGGAA-1,Hippo Pvalb GABA-0,Hippo Pvalb GABA-0-0,Hippo Pvalb GABA-0-1
MC39C_HIP:AACAGTCCATCCCTTG-1,Hippo Pvalb GABA-1,Hippo Pvalb GABA-1-0,NaN
MC39C_HIP:AACGTACCATCCTGGG-1,Hippo Pvalb GABA-0,Hippo Pvalb GABA-0-0,Hippo Pvalb GABA-0-0
...,...,...,...
MW51A_AMY:TTTGGTTCAATCACTG-1,AMY Adamtsl1 Glut-0,AMY Adamtsl1 Glut-0-0,AMY Adamtsl1 Glut-0-0
MW51A_AMY:TTTGGTTGTTGCAGAG-1,AMY Adamtsl1 Glut-2,AMY Adamtsl1 Glut-2-0,AMY Adamtsl1 Glut-2-0
MW51A_AMY:TTTGGTTTCTTAAGCG-1,AMY Adamtsl1 Glut-1,AMY Adamtsl1 Glut-1-0,AMY Adamtsl1 Glut-1-0
MW51A_AMY:TTTGTGTCAAAGCTTC-1,AMY Adamtsl1 Glut-3,AMY Adamtsl1 Glut-3-0,AMY Adamtsl1 Glut-3-0


In [74]:
df_merge.to_csv("/data2st1/junyi/output/atac0416/iterative/annotated_l3l4.csv")